In [47]:
from Bio import Entrez
from Bio import SeqIO
import re
import os
import subprocess

keyword="phage.*[genome|genomic sequence|complete|!partial]"
accs=[]
titles=[]
acdict={}

splist=open("../NCBI_phage_genome_search.txt").readlines()
for i in range(1,len(splist)/160,4):
    if re.search(keyword, splist[i]):
        name=splist[i].split(",")[0].split(".")[1]
        name2=re.sub(r"complete genome\n|genomic sequence\n|\n","", name)
        if name not in titles:
            titles.append(name2)
            acc=splist[i+2].split(" ")[0]
            accs.append(acc)
            acdict[acc]=name2
accs_str=",".join(accs)
    

os.mkdir("pgenomes")
Entrez.email="julia.brown@einstein.yu.edu"
for a in accs:
    handle=Entrez.efetch(db="nucleotide", id=str(a), rettype="gb", retmode="text")
    filename="./pgenomes/"+a+".gbk"
    out=open(filename, "w")
    out.write(handle.read())
    out.close()
    handle.close()
    SeqIO.convert("./pgenomes/"+a+".gbk","gb","./pgenomes/"+a+".fasta","fasta")
    print(a+" converted")



from Bio.SeqUtils import GC

out=open("phage.genome.info.txt","w")

taxdict={}

for phage in accs:
    gbk=SeqIO.parse("./pgenomes/"+phage+".gbk", "genbank")
    for rec in gbk:
        taxonomy=rec.annotations["taxonomy"]
        if len(taxonomy)>4:
            tax=taxonomy[3]
        else:
            tax=taxonomy[-1]
        taxdict[phage]=tax
        gc_content=GC(rec.seq)
        out.write(phage+"\t"+acdict[phage]+"\t"+tax+"\t"+str(len(rec.seq))+"\t"+str(gc_content)+"\n")
out.close()

os.mkdir("./pgenomes/trna/")

args=[]
for phage in accs:
    args=("tRNAscan-SE -o ./pgenomes/trna/"+phage+".trnas.txt -G -D ./pgenomes/"+phage+".fasta")
    subprocess.call(args.split(" "))
    print("scanning phage "+ phage +" for tRNAs!")
tRNA_phages=[]


for n in accs:
    tfile="./pgenomes/trna/"+n+".trnas.txt"
    if os.path.getsize(tfile)>0:
        tRNA_phages.append(n)
    
print("there are a total of "+str(len(accs))+" phages but only "+str(len(tRNA_phages))+" have tRNAs")

out=open("phage_tRNA_summary.txt","w")
out.write("phage_short\tphage_full\tbound1\tbound2\ttRNA\tanti_codon\tintron_bound1\titron_bound2\tscore\n")

for phage in tRNA_phages:
    info=open("./pgenomes/trna/"+phage+".trnas.txt").readlines()
    for line in info[3:]:
        vec=line.split("\t")
        out.write(acdict[phage]+"\t"+vec[0]+"\t"+vec[2]+"\t"+vec[3]+"\t"+vec[4]+"\t"+vec[5]+"\t"+vec[6]+"\t"+vec[7]+"\t"+vec[8])
out.close()

print("done!")

KF582788.2 converted
HQ630627.1 converted
NC_024794.1 converted
JX080304.2 converted
JX080305.2 converted
JX080303.2 converted
JX080302.2 converted
JX080301.2 converted
JX080300.2 converted
EU418428.2 converted
NC_020083.1 converted
NC_019398.1 converted
NC_020080.1 converted
NC_019505.1 converted
NC_015457.1 converted
NC_014595.1 converted
KR014248.1 converted
NC_027337.1 converted
NC_004914.2 converted
NC_023693.1 converted
NC_019400.1 converted
scanning phage KF582788.2 for tRNAs!
scanning phage HQ630627.1 for tRNAs!
scanning phage NC_024794.1 for tRNAs!
scanning phage JX080304.2 for tRNAs!
scanning phage JX080305.2 for tRNAs!
scanning phage JX080303.2 for tRNAs!
scanning phage JX080302.2 for tRNAs!
scanning phage JX080301.2 for tRNAs!
scanning phage JX080300.2 for tRNAs!
scanning phage EU418428.2 for tRNAs!
scanning phage NC_020083.1 for tRNAs!
scanning phage NC_019398.1 for tRNAs!
scanning phage NC_020080.1 for tRNAs!
scanning phage NC_019505.1 for tRNAs!
scanning phage NC_015457.

In [68]:
pwd

u'/Users/jmb/Desktop/ViralFate/tRNA_info'

In [67]:
!scp collect_trna_info.py NCBI_phage_genome_search.txt jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/

collect_trna_info.py                          100% 1177     1.2KB/s   00:00    
NCBI_phage_genome_search.txt                  100%  560KB 559.7KB/s   00:00    


In [65]:
!scp bulk_phage_trna_search.slurm jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/

bulk_phage_trna_search.slurm                  100%  205     0.2KB/s   00:00    


In [69]:
!scp jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/phage_tRNA_summary.txt ./bulk_phage_trna_summary.txt

phage_tRNA_summary.txt                        100%  434KB 434.3KB/s   00:00    


In [73]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/tRNA/phage.genome.info.txt ./phage.genome.info.txt

phage.genome.info.txt                         100%  229KB 228.7KB/s   00:00    


In [20]:
import re

from Bio import SeqIO


os.chdir("/Users/jmb/Desktop/ViralFate/tRNA_info/")

In [5]:
keyword="phage.*[genome|genomic sequence|complete|!partial]"
accs=[]
titles=[]
acdict={}

splist=open("./data/NCBI_phage_genome_search.txt").readlines()
for i in range(1,len(splist),4):
    if re.search(keyword, splist[i]):
        name=splist[i].split(",")[0].split(".")[1]
        name2=re.sub(r"complete genome\n|genomic sequence\n|\n","", name)
        if name not in titles:
            titles.append(name2)
            acc=splist[i+2].split(" ")[0]
            accs.append(acc)
            acdict[acc]=name2
accs_str=",".join(accs)

In [54]:
out=open("large_acc_list.txt","w")
out.write(accs_str)
out.close()

In [58]:
from Bio import SeqIO
import re
import os

accs=open("large_acc_list.txt").read().split(",")

#out=open("phage_publication_info.txt","w")
#out.write("acc_num\tphage_name\tref_title\tpubmed_id\n")
for phage in accs:
    gbk_file="./test/pgenomes/"+phage+".gbk"
    if os.path.exists(gbk_file):
        gbk=SeqIO.parse(gbk_file, "genbank")
        for rec in gbk:
            ref_title=str(rec.annotations['references'][0]).split("\n")
            for i in ref_title:
                if i.startswith("title"):
                    TITLE=i
                elif i.startswith("pubmed id:"):
                    ID=i
            #out.write(phage+"\t"+rec.annotations['source']+"\t"+TITLE.replace("title:","")+"\t"+ID.replace("pubmed id:","")+"\n")
#out.close()

3144
KF582788.2	Escherichia phage vB_EcoM_JS09	 Isolation and characterization of a novel lytic T4-like coliphage vB_EcoM_JS09 infecting APEC	 

HQ630627.1	Pseudomonas phage PhiPA3	 The Pseudomonas aeruginosa generalized transducing phage phiPA3 is a new member of the phiKZ-like group of 'jumbo' phages, and infects model laboratory strains and clinical isolates from cystic fibrosis patients	 21163841

NC_024794.1	Escherichia phage vB_EcoM_PhAPEC2	 A cocktail of in vitro efficient phages is not a guarantee for in vivo therapeutic results against avian colibacillosis	 24269008

JX080304.2	Staphylococcus phage MSA6	 Genomics of Staphylococcal Twort-like Phages - Potential Therapeutics of the Post-Antibiotic Era	 22748811

JX080305.2	Staphylococcus phage P4W	 Genomics of Staphylococcal Twort-like Phages - Potential Therapeutics of the Post-Antibiotic Era	 22748811

JX080303.2	Staphylococcus phage Fi200W	 Genomics of Staphylococcal Twort-like Phages - Potential Therapeutics of the Post-Anti

In [23]:
from Bio import SeqIO
import re
import os

#out=open("phage_publication_info.txt","w")
#out.write("acc_num\tphage_name\tref_title\tpubmed_id\n")
for phage in accs:
    gbk_file="./data/test/pgenomes/"+phage+".gbk"
    if os.path.exists(gbk_file):
        handle=SeqIO.parse(gbk_file, "genbank")
        for g in handle:
            for feature in g.features:
                if feature.type=="source":
                    if "host" in feature.qualifiers.keys():
                        print(feature.qualifiers["host"][0])
                    elif "lab_host" in feature.qualifiers.keys():
                        print(feature.qualifiers["lab_host"][0]+"***")
                    else:
                        print("no host specified for "+phage)

avian pathogenic Escherichia coli and genetically engineered Escherichia coli B and K strain
Pseudomonas aeruginosa strain PAO1
Escherichia coli***
Staphylococcus aureus subsp. aureus strain ATCC 25923***
Staphylococcus aureus strain 6409***
Staphylococcus aureus strain 6409***
Staphylococcus aureus strain Z11778***
Staphylococcus aureus strain R19930***
Staphylococcus aureus strain NCTC9789 (=PS80)***
Staphylococcus aureus NCTC9789 (=PS80)
Serratia sp.
Cronobacter sakazakii
Klebsiella pneumoniae
Escherichia coli
Shigella flexneri
Shigella sonnei
Escherichia coli O145:NM***
Escherichia coli B strains
Escherichia coli O157:H7
Escherichia coli K92
Cronobacter sakazakii


In [59]:
!scp large_acc_list.txt jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/

large_acc_list.txt                            100%   35KB  35.1KB/s   00:00    


In [60]:
!scp extract_pub_titles.py jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/

extract_pub_titles.py                         100%  763     0.8KB/s   00:00    


In [62]:
!scp title_extract.slurm jbrown@eofe5.mit.edu:/nobackup1/jbrown/tRNA/

title_extract.slurm                           100%  169     0.2KB/s   00:00    
